In [101]:
import sqlite3

stocks = [
    ('GOOG', 100, 490.1),
    ('AAPL', 50, 545.75),
    ('FB', 150, 7.45),
    ('HPQ', 75, 33.2),
]

db = sqlite3.connect('database.db')
c = db.cursor()
c.execute('create table portfolio (symbol text, shares integer, price real)')
db.commit()

In [102]:
c.executemany('insert into portfolio values (?,?,?)', stocks)
db.commit()

In [103]:
for row in db.execute('select * from portfolio'):
    print(row)

('GOOG', 100, 490.1)
('AAPL', 50, 545.75)
('FB', 150, 7.45)
('HPQ', 75, 33.2)


In [104]:
min_price = 100
for row in db.execute('select * from portfolio where price >= ?', (min_price,)):
    print(row)

('GOOG', 100, 490.1)
('AAPL', 50, 545.75)


In [111]:
import binascii
import base64

# Изначальная байтовая строка
s = b'hello'

# Закодировать в hex
h = binascii.b2a_hex(s)
print(h)

# Декодировать обратно в байты
print(binascii.a2b_hex(h))

# base64
h = base64.b16encode(s)
print(h)
print(base64.b16decode(h))
print(h.decode('ascii'))

# Закодировать в Base64
a = base64.b64encode(s)
print(a)

# Декодировать из Base64
print(base64.b64decode(a))

b'68656c6c6f'
b'hello'
b'68656C6C6F'
b'hello'
68656C6C6F
b'aGVsbG8='
b'hello'


In [1]:
from struct import Struct

def write_records(records, format, f):
    '''Записывает последовательность кортежей в бинарный файл структур.'''
    record_struct = Struct(format)
    for r in records:
        f.write(record_struct.pack(*r))
        
records = [(1, 2.3, 4.5),
           (6, 7.8, 9.0),
           (12, 13.4, 56.7)]

with open('data.b', 'wb') as f:
    write_records(records, '<idd', f)

In [2]:
def read_records(format, f):
    record_struct = Struct(format)
    chunks = iter(lambda: f.read(record_struct.size), b'')
    return (record_struct.unpack(chunk) for chunk in chunks)

with open('data.b', 'rb') as f:
    for rec in read_records('<idd', f):
        print(rec)

(1, 2.3, 4.5)
(6, 7.8, 9.0)
(12, 13.4, 56.7)


In [3]:
def unpack_records(format, data):
    record_struct = Struct(format)
    return (record_struct.unpack_from(data, offset) for offset in range(0, len(data), record_struct.size))

with open('data.b', 'rb') as f:
    data = f.read()
    
for rec in unpack_records('<idd', data):
    print(rec)

(1, 2.3, 4.5)
(6, 7.8, 9.0)
(12, 13.4, 56.7)


In [7]:
record_struct = Struct('<idd')
print(record_struct.size)
print(record_struct.pack(1, 2.0, 3.0))
print(record_struct.unpack(record_struct.pack(1, 2.0, 3.0)))

20
b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@'
(1, 2.0, 3.0)


In [10]:
with open('data.b', 'rb') as f:
    chunks = iter(lambda: f.read(20), b'')
    print(chunks)

    for chk in chunks:
        print(chk)

b'\x01\x00\x00\x00ffffff\x02@\x00\x00\x00\x00\x00\x00\x12@'
b'\x06\x00\x00\x00333333\x1f@\x00\x00\x00\x00\x00\x00"@'
b'\x0c\x00\x00\x00\xcd\xcc\xcc\xcc\xcc\xcc*@\x9a\x99\x99\x99\x99YL@'


In [14]:
from collections import namedtuple

def read_records(format, f):
    record_struct = Struct(format)
    while True:
        chk = f.read(record_struct.size)
        if chk == b'':
            break
        yield record_struct.unpack(chk)
    return records

def unpack_records(format, data):
    record_struct = Struct(format)
    return (record_struct.unpack(data[offset:offset + record_struct.size]) for offset in range(0, len(data), record_struct.size))

Record = namedtuple('Record', ['kind','x','y'])

with open('data.b', 'rb') as f:
    records = (Record(*r) for r in read_records('<idd', f))
    
    for r in records:
        print(r.kind, r.x, r.y)


1 2.3 4.5
6 7.8 9.0
12 13.4 56.7


In [15]:
import numpy as np

with open('data.b', 'rb') as f:
    records = np.fromfile(f, dtype='<i,<d,<d')
    print(records)

[( 1,  2.3,  4.5) ( 6,  7.8,  9. ) (12, 13.4, 56.7)]


In [17]:
import struct
import itertools


polys = [
    [ (1.0, 2.5), (3.5, 4.0), (2.5, 1.5) ],
    [ (7.0, 1.2), (5.1, 3.0), (0.5, 7.5), (0.8, 9.0) ],
    [ (3.4, 6.3), (1.2, 0.5), (4.6, 9.2) ],
]

def write_polys(filename, polys):
    # Определяем ограничивающий параллелепипед
    flattened = list(itertools.chain(*polys))
    min_x = min(x for x, y in flattened)
    max_x = max(x for x, y in flattened)
    min_y = min(y for x, y in flattened)
    max_y = max(y for x, y in flattened)
    
    with open(filename, 'wb') as f:
        f.write(struct.pack('<iddddi', 0x1234, min_x, min_y, max_x, max_y, len(polys)))
        
        for poly in polys:
            size = len(poly) * struct.calcsize('<dd')
            f.write(struct.pack('<i', size+4))
            for pt in poly:
                f.write(struct.pack('<dd', *pt))
            
# Вызываем с нашими данными полигонов
write_polys('polys.bin', polys)

In [18]:
def read_polys(filename):
    with open(filename, 'rb') as f:
        # Читаем заголовок
        header = f.read(40)
        file_code, min_x, min_y, max_x, max_y, num_polys = struct.unpack('<iddddi', header)
        
        polys = []
        for n in range(num_polys):
            pbytes, = struct.unpack('<i', f.read(4))
            poly = []
            for m in range(pbytes // 16):
                pt = struct.unpack('<dd', f.read(16))
                poly.append(pt)
            polys.append(poly)
    
    return polys

read_polys('polys.bin')

[[(1.0, 2.5), (3.5, 4.0), (2.5, 1.5)],
 [(7.0, 1.2), (5.1, 3.0), (0.5, 7.5), (0.8, 9.0)],
 [(3.4, 6.3), (1.2, 0.5), (4.6, 9.2)]]

In [20]:
class StructField:
    '''Дескриптор, представляющий простое поле структуры'''
    def __init__(self, format, offset):
        self.format = format
        self.offset = offset
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            r = struct.unpack_from(self.format, instance._buffer, self.offset)
        return r[0] if len(r) == 1 else r
    

class Structure:
    def __init__(self, bytedata):
        self._buffer = memoryview(bytedata)
        
class PolyHeader(Structure):
    file_code = StructField('<i', 0)
    min_x = StructField('<d', 4)
    min_y = StructField('<d', 12)
    max_x = StructField('<d', 20)
    max_y = StructField('<d', 28)
    num_polys = StructField('<i', 36)
    
with open('polys.bin', 'rb') as f:
    phead = PolyHeader(f.read(40))
    print(phead.file_code == 0x1234)
    print(phead.min_x)
    print(phead.min_y)
    print(phead.max_x)
    print(phead.max_y)
    print(phead.num_polys)

True
0.5
0.5
7.0
9.2
3


In [21]:
class StructureMeta(type):
    '''Метакласс, который автоматически создает дескрипторы StructField'''
    def __init__(self, clsname, bases, clsdict):
        fields = getattr(self, '_fields_', [])
        byte_order = ''
        offset = 0
        
        for format, fieldname in fields:
            if format.startswith(('<','>','!','@')):
                byte_order = format[0]
                format = format[1:]
            format = byte_order + format
            setattr(self, fieldname, StructField(format, offset))
            offset += struct.calcsize(format)
        setattr(self, 'struct_size', offset)
        

class Structure(metaclass=StructureMeta):
    def __init__(self, bytedata):
        self._buffer = bytedata
        
    @classmethod
    def from_file(cls, f):
        return cls(f.read(cls.struct_size))
    

class PolyHeader(Structure):
    _fields_ = [
        ('<i', 'file_code'),
        ('d', 'min_x'),
        ('d', 'min_y'),
        ('d', 'max_x'),
        ('d', 'max_y'),
        ('i', 'num_polys')
    ]
    

with open('polys.bin', 'rb') as f:
    phead = PolyHeader(f.read(40))
    print(phead.file_code == 0x1234)
    print(phead.min_x)
    print(phead.min_y)
    print(phead.max_x)
    print(phead.max_y)
    print(phead.num_polys)    
    
    
    

True
0.5
0.5
7.0
9.2
3


In [30]:
class SizedRecord:
    def __init__(self, bytedata):
        self._buffer = memoryview(bytedata)
        
    @classmethod
    def from_file(cls, f, size_fmt, includes_size=True):
        sz_nbytes = struct.calcsize(size_fmt)
        sz_bytes = f.read(sz_nbytes)
        sz, = struct.unpack(size_fmt, sz_bytes)
        buf = f.read(sz - includes_size * sz_nbytes)
        return cls(buf)
    
    def iter_as(self, code):
        if isinstance(code, str):
            s = struct.Struct(code)
            for off in range(0, len(self._buffer), s.size):
                yield s.unpack_from(self._buffer, off)
        elif isinstance(code, StructureMeta):
            size = code.struct_size
            for off in range(0, len(self._buffer), size):
                data = self._buffer[off:off+size]
                yield code(data)
                

with open('polys.bin', 'rb') as f:
    phead = PolyHeader.from_file(f)
    print(phead.num_polys)
    
    polydata = [SizedRecord.from_file(f, '<i') for n in range(phead.num_polys)]
    print(polydata)
    
    for n, poly in enumerate(polydata):
        print('Polygon', n)
        for p in poly.iter_as('<dd'):
            print(p)

3
[<__main__.SizedRecord object at 0x000001B3FF518700>, <__main__.SizedRecord object at 0x000001B3FF620D00>, <__main__.SizedRecord object at 0x000001B3FF620AF0>]
Polygon 0
(1.0, 2.5)
(3.5, 4.0)
(2.5, 1.5)
Polygon 1
(7.0, 1.2)
(5.1, 3.0)
(0.5, 7.5)
(0.8, 9.0)
Polygon 2
(3.4, 6.3)
(1.2, 0.5)
(4.6, 9.2)


In [5]:
'''Строки формата Unicode'''
import unicodedata

def unicode_test(value):
    name = unicodedata.name(value)
    value2 = unicodedata.lookup(name)
    print(f"value={value}, name={name}, value2={value2}")

unicode_test('A')
unicode_test('$')
unicode_test('\u00a2')
unicode_test('\u20ac')
unicode_test('\u2603')

value=A, name=LATIN CAPITAL LETTER A, value2=A
value=$, name=DOLLAR SIGN, value2=$
value=¢, name=CENT SIGN, value2=¢
value=€, name=EURO SIGN, value2=€
value=☃, name=SNOWMAN, value2=☃


In [21]:
import unicodedata

a = 'pýtĥöñ is awesome\n'
b = unicodedata.normalize('NFD', a)
print(b.encode('ascii', 'ignore').decode('ascii'))

python is awesome



In [11]:
'''Кодирование и декодирование с помощью кодировки UTF-8'''
snowman = '\u2603'

# Кол-во символов
print(len(snowman))

# Кодирование, последовательностью байтов
ds = snowman.encode('utf-8')
# Использовано три байта для того, чтобы закодировать один символ snowman
print(len(ds))
print(ds)

# Используйте значение 'replace', чтобы заменить неизвестные символы символами ?
print(snowman.encode('ascii', 'replace'))

# Используйте значение 'backslashreplace', чтобы создать строку,
# содержащую символы Python Unicode вроде unicode-escape
print(snowman.encode('ascii', 'backslashreplace'))

1
3
b'\xe2\x98\x83'
b'?'
b'\\u2603'


In [20]:
place = 'caf\u00e9'
print(place)
print(type(place))

print('-' * 20)

place_bytes = place.encode('utf-8')
print(place_bytes)
print(type(place_bytes))

place2 = place_bytes.decode('utf-8')
print(place2)

print('-' * 20)

place4 = place_bytes.decode('latin-1')
print(place4)
place5 = place_bytes.decode('windows-1252')
print(place5)

café
<class 'str'>
--------------------
b'caf\xc3\xa9'
<class 'bytes'>
café
--------------------
cafÃ©
cafÃ©


In [6]:
'''bytes и bytearray'''

blist = [1, 2, 3, 255]

# кортеж байтов
the_bytes = bytes(blist)
print(the_bytes)

# список байтов
the_byte_array = bytearray(blist)
print(the_byte_array)

the_byte_array[1] = 127
print(the_byte_array)

print('-' * 50)

# Каждая из этих переменных может содержать результат,
# состоящий из 256 элементов, имеющих значения от 0 до 255
the_bytes = bytes(range(0, 256))
the_byte_array = bytearray(range(0, 256))
print(the_bytes)
print(the_byte_array)


b'\x01\x02\x03\xff'
bytearray(b'\x01\x02\x03\xff')
bytearray(b'\x01\x7f\x03\xff')
--------------------------------------------------
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\x7f\x80\x81\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x8b\x8c\x8d\x8e\x8f\x90\x91\x92\x93\x94\x95\x96\x97\x98\x99\x9a\x9b\x9c\x9d\x9e\x9f\xa0\xa1\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xab\xac\xad\xae\xaf\xb0\xb1\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xbb\xbc\xbd\xbe\xbf\xc0\xc1\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xcb\xcc\xcd\xce\xcf\xd0\xd1\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xdb\xdc\xdd\xde\xdf\xe0\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xeb\xec\xed\xee\xef\xf0\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xfb\xfc\xfd\xfe\xff'
bytearray(b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x

In [10]:
'''Преобразуем бинарные данные с помощью модуля struct'''
import struct

valid_png_header = b'\x89PNG\r\n\x1a\n'

data = b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\00\x00\x00\x9a\x00\x00\x00\x8d\x08\x02\x00\x00\x00\xc0'
if data[:8] == valid_png_header:
    width, height = struct.unpack('>LL', data[16:24])
    print(f'Valid PNG, width: {width} height: {height}')
else:
    print('Not a valid PNG')
    

Valid PNG, width: 154 height: 141


In [5]:
'''Запись в текстовый файл с помощью функции write()'''

poem = '''There was a young lady named Bright,
Whose speed was far faster than light
She started one day
In a relative way,
And returned on the previous night.
'''

len(poem)

fout = open('relativity', 'wt')
# fout.write(poem)
print(poem, file=fout) # запись с помощью print
fout.close()

In [6]:
poem = '''There was a young lady named Bright,
Whose speed was far faster than light
She started one day
In a relative way,
And returned on the previous night.
'''

# Если исходная строка большая, вы можете записывать в файл ее фрагменты до тех пор,
# пока не запишете ее всю
fout = open('relativity', 'wt')
size = len(poem)
offset = 0
chunk = 100
while True:
    if offset > size:
        break
    fout.write(poem[offset:offset+chunk])
    offset += chunk
fout.close()

In [11]:
'''Считываем данные из текстового файла с помощью функций read(), readline() и readlines()'''
fin = open('relativity', 'rt')
poem = fin.read()
fin.close()

print(poem)

There was a young lady named Bright,
Whose speed was far faster than light
She started one day
In a relative way,
And returned on the previous night.



In [10]:
# Считывание по 100 символов за раз и присоединение 
# каждого фрагмента к строке poem, чтобы воссоздать оригинал
poem = ''
fin = open('relativity', 'rt')
chunk = 100
while True:
    fragment = fin.read(chunk)
    if not fragment:
        break
    poem += fragment
fin.close()

print(poem)

There was a young lady named Bright,
Whose speed was far faster than light
She started one day
In a relative way,
And returned on the previous night.



In [12]:
# Присоединение каждой строки к строке poem, чтобы воссоздать оригинал
poem = ''
fin = open('relativity', 'rt')
while True:
    line = fin.readline()
    if not line:
        break
    poem += line
fin.close()

print(poem)

There was a young lady named Bright,
Whose speed was far faster than light
She started one day
In a relative way,
And returned on the previous night.



In [17]:
# Использование итератора. Возвращает по одной строке за раз.
poem = ''
fin = open('relativity', 'rt')
for line in fin:
    poem += line
fin.close()

print(poem)

There was a young lady named Bright,
Whose speed was far faster than light
She started one day
In a relative way,
And returned on the previous night.



In [21]:
# Функция readlines() считывает по одной строке за раз
# и возвращает список этих строк
fin = open('relativity', 'rt')
lines = fin.readlines()
fin.close()
print(len(lines), 'lines read')

for line in lines:
    print(line, end='')

5 lines read
There was a young lady named Bright,
Whose speed was far faster than light
She started one day
In a relative way,
And returned on the previous night.


In [22]:
'''Записываем данные в бинарный файл с помощью функции write()'''

bdata = bytes(range(0, 256))
fout = open('bfile', 'wb')
fout.write(bdata)
fout.close()

In [23]:
# Запись бинарных данных фрагментами
bdata = bytes(range(0, 256))
fout = open('bfile', 'wb')
size = len(poem)
offset = 0
chunk = 100
while True:
    if offset > size:
        break
    fout.write(bdata[offset:offset+chunk])
    offset += chunk
fout.close()

In [24]:
'''Читаем бинарные файлы с помощью функции read()'''
fin = open('bfile', 'rb')
bdata = fin.read()
fin.close()
print(bdata)

b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\x7f\x80\x81\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x8b\x8c\x8d\x8e\x8f\x90\x91\x92\x93\x94\x95\x96\x97\x98\x99\x9a\x9b\x9c\x9d\x9e\x9f\xa0\xa1\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xab\xac\xad\xae\xaf\xb0\xb1\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xbb\xbc\xbd\xbe\xbf\xc0\xc1\xc2\xc3\xc4\xc5\xc6\xc7'


In [24]:
import yaml

with open('mcintyre.yaml', 'rt', encoding='utf-8') as fin:
    text = fin.read()  
    data = yaml.safe_load(text)
    print(data['details'])

{'bearded': True, 'themes': ['cheese', 'Canada']}


In [25]:
import configparser

cfg = configparser.ConfigParser()
cfg.read('settings.cfg')

['settings.cfg']

In [30]:
import pickle

class Tiny:
    def __str__(self):
        return 'tiny'
    
obj1 = Tiny()
print(obj1) 

pickled = pickle.dumps(obj1)
print(pickled)

obj2 = pickle.loads(pickled)
print(obj2)   

tiny
b'\x80\x04\x95\x18\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x04Tiny\x94\x93\x94)\x81\x94.'
tiny


In [9]:
'''Хранилища данных NoSQL'''
import dbm

with dbm.open('definitions', 'c') as db:
    db['mustard'] = 'yellow'
    db['ketchup'] = 'red'
    db['pesto'] = 'green'
    
    
with dbm.open('definitions', 'r') as db:
    for i in db:
        print(i)



b'mustard'
b'ketchup'
b'pesto'
